In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import torch
import gc
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM

In [3]:
models = [
    "HuggingFaceTB/SmolLM-135M-Instruct",
    "meta-llama/Llama-3.2-3B-Instruct",
    "meta-llama/Llama-3.1-8B-Instruct",
]


In [4]:
for model_name in models:
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
    number_of_total_parameters = sum(p.numel() for p in model.parameters())
    count_of_outliers = 0
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            try:
                sorted_weights, _ = torch.sort(module.weight.data.view(-1))
                k = int(0.25 * len(sorted_weights))
                lower_quartile = sorted_weights[k]
                upper_quartile = sorted_weights[-k]
                iqr = upper_quartile - lower_quartile
                lower_bound = lower_quartile - 1.5 * iqr
                upper_bound = upper_quartile + 1.5 * iqr
                outliers = module.weight.data[
                    (module.weight.data < lower_bound)
                    | (module.weight.data > upper_bound)
                ]
                count_of_outliers += len(outliers)
            except:
                print(name)

    print(f"{model_name}: Total number of outliers: {count_of_outliers}")
    print(f"{model_name}: Total number of parameters: {number_of_total_parameters}")
    print(
        f"{model_name}: Percentage of outliers: {count_of_outliers / number_of_total_parameters * 100:.2f}%"
    )
    del model
    gc.collect()
    torch.cuda.empty_cache()

HuggingFaceTB/SmolLM-135M-Instruct: Total number of outliers: 1983917
HuggingFaceTB/SmolLM-135M-Instruct: Total number of parameters: 134515008
HuggingFaceTB/SmolLM-135M-Instruct: Percentage of outliers: 1.47%


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

meta-llama/Llama-3.2-3B-Instruct: Total number of outliers: 45126190
meta-llama/Llama-3.2-3B-Instruct: Total number of parameters: 3212749824
meta-llama/Llama-3.2-3B-Instruct: Percentage of outliers: 1.40%


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

lm_head
meta-llama/Llama-3.1-8B-Instruct: Total number of outliers: 90763642
meta-llama/Llama-3.1-8B-Instruct: Total number of parameters: 8030261248
meta-llama/Llama-3.1-8B-Instruct: Percentage of outliers: 1.13%
